In [60]:
import json
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import pickle

In [61]:
# Load the intents from a JSON file
with open('intents.json') as file:
    data = json.load(file)

In [62]:
# Initialize lists for training data
training_sentences = []
training_labels = []
labels = []
responses = []

In [63]:
# Extract data from intents.json
for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])

    if intent['tag'] not in labels:
        labels.append(intent['tag'])

num_classes = len(labels)

In [64]:
# Encode labels
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [65]:
# Tokenize and pad the training data
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

In [66]:
# adding out of vocabulary token
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token) 
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [67]:
# Build the model
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

In [68]:
# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [69]:
# Display model architecture
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d_4  (None, 16)                0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dense_12 (Dense)            (None, 16)                272       
                                                                 
 dense_13 (Dense)            (None, 16)                272       
                                                                 
 dense_14 (Dense)            (None, 30)                510       
                                                                 
Total params: 17054 (66.62 KB)
Trainable params: 17054 (66.62 KB)
Non-trainable params: 0 (0.00 Byte)
__________________

In [70]:
# Train the model
epochs = 550
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)
model.compile(loss='mse', optimizer='adam')

Epoch 1/550


3/3 [==============================] - 0s 2ms/step - loss: 3.4028 - accuracy: 0.0366
Epoch 2/550
3/3 [==============================] - 0s 2ms/step - loss: 3.4008 - accuracy: 0.0366
Epoch 3/550
3/3 [==============================] - 0s 2ms/step - loss: 3.3996 - accuracy: 0.0488
Epoch 4/550
3/3 [==============================] - 0s 1ms/step - loss: 3.3986 - accuracy: 0.0610
Epoch 5/550
3/3 [==============================] - 0s 2ms/step - loss: 3.3976 - accuracy: 0.0488
Epoch 6/550
3/3 [==============================] - 0s 2ms/step - loss: 3.3967 - accuracy: 0.0366
Epoch 7/550
3/3 [==============================] - 0s 2ms/step - loss: 3.3958 - accuracy: 0.0366
Epoch 8/550
3/3 [==============================] - 0s 2ms/step - loss: 3.3948 - accuracy: 0.0366
Epoch 9/550
3/3 [==============================] - 0s 2ms/step - loss: 3.3937 - accuracy: 0.0488
Epoch 10/550
3/3 [==============================] - 0s 2ms/step - loss: 3.3928 - accuracy: 0.0732
Epoch 11/550
3/3 [=======================

In [71]:
#Saving Model
model.save('chat_model')

INFO:tensorflow:Assets written to: chat_model/assets


INFO:tensorflow:Assets written to: chat_model/assets


In [72]:
#Saving tokenizer
with open('tokenizer.pickle' ,'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [73]:
#Saving label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)